In [1]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from skimage import feature
from skimage import color
from scipy.misc import imresize,imsave

In [3]:
root_path = "C:\Users\Patsorn\Downloads\siggraph_maybe/"
photo_path = root_path + "photos"
output_fold = root_path+ "list"

In [4]:
with open(output_fold+'/cat.txt','r') as my_file:
    cat_list = [c.rstrip() for c in my_file.readlines()]
with open(output_fold+'/image_list.txt','r') as my_file:
    img_list = [c.rstrip() for c in my_file.readlines()]
with open(output_fold+'/invalid_photo/invalid.txt','r') as my_file:
    invalid = [c.rstrip() for c in my_file.readlines()]
with open(output_fold+'/invalid_photo/ambiguous.txt','r') as my_file:
    amb = [c.rstrip() for c in my_file.readlines()]
with open(output_fold+'/invalid_photo/context.txt','r') as my_file:
    cont = [c.rstrip() for c in my_file.readlines()]
with open(output_fold+'/invalid_photo/perspective.txt','r') as my_file:
    persp = [c.rstrip() for c in my_file.readlines()]
with open(output_fold+'/test_img_list.txt','r') as my_file:
    test_img_list = [c.rstrip() for c in my_file.readlines()]
with open(output_fold+'/train_img_list.txt','r') as my_file:
    train_img_list = [c.rstrip() for c in my_file.readlines()]

In [26]:
#gen cat list, only need to run once
print "Generating category list--", 
cat_list = os.listdir(photo_path)

f = open(output_fold+'/cat_id.txt','w')
for i, cat in enumerate(cat_list):
    f.write(cat+' '+str(i)+'\n')
f.close()
print 'cat_id.txt'
print "Generating category list--",
cat_list = os.listdir(photo_path)

f = open(output_fold+'/cat.txt','w')
for i, cat in enumerate(cat_list):
    f.write(cat+'\n')
f.close()
print 'cat.txt'

Generating category list-- cat_id.txt
Generating category list-- cat.txt


In [16]:
#load up cat list
#os.listdir(photo_path) would work as well
with open(output_fold+'/cat.txt','r') as my_file:
    cat_list = [c.rstrip() for c in my_file.readlines()]

In [25]:
#generate invalid list
annotations = ['Invalid/error','Unidentifiable/ambiguous','Context','Perspective']
dict_ann = {}
for i,ann in enumerate(annotations):
    dict_ann[ann] = i
ann_fold = root_path +'svg_ann'
invalid_fold = output_fold+'/invalid_sketch'
print "Generating invalid list of sketches--", 
cur_mode = -1
f_invalid = open(invalid_fold+'/invalid.txt','w')
f_amb = open(invalid_fold+'/ambiguous.txt','w')
f_cont = open(invalid_fold+'/context.txt','w')
f_persp = open(invalid_fold+'/perspective.txt','w')
for i, cat in enumerate(cat_list):
    if os.path.isfile(ann_fold+'/'+cat+'/invalid.txt'):
        fr = open(ann_fold+'/'+cat+'/invalid.txt','r')
        for line in fr:
            if line.rstrip() in annotations:
                cur_mode = dict_ann[line.rstrip()]
            elif (not cur_mode == -1) and (line.startswith('n')):
                if cur_mode==0:
                    f_invalid.write(cat+'/'+line.rstrip()+'.png\n')
                elif cur_mode == 1:
                    f_amb.write(cat+'/'+line.rstrip()+'.png\n')
                elif cur_mode == 2:
                    f_cont.write(cat+'/'+line.rstrip()+'.png\n')
                elif cur_mode == 3:
                    f_persp.write(cat+'/'+line.rstrip()+'.png\n')               
        fr.close()
f_invalid.close()
f_amb.close()
f_cont.close()
f_persp.close()
print '/invalid.txt'

Generating invalid list of sketches-- /invalid.txt


In [30]:
#generate invalid list of photos
annotations = ['Invalid/error','Unidentifiable/ambiguous','Context','Perspective']
dict_ann = {}
for i,ann in enumerate(annotations):
    dict_ann[ann] = i
ann_fold = root_path +'svg_ann'
invalid_fold = output_fold+'/invalid_photo'
print "Generating invalid list of photos--", 
cur_mode = -1
invalid = []
amb = []
cont = []
persp = []
f_invalid = open(invalid_fold+'/invalid.txt','w')
f_amb = open(invalid_fold+'/ambiguous.txt','w')
f_cont = open(invalid_fold+'/context.txt','w')
f_persp = open(invalid_fold+'/perspective.txt','w')
for i, cat in enumerate(cat_list):
    if os.path.isfile(ann_fold+'/'+cat+'/invalid.txt'):
        fr = open(ann_fold+'/'+cat+'/invalid.txt','r')
        for line in fr:
            if line.rstrip() in annotations:
                cur_mode = dict_ann[line.rstrip()]
            elif (not cur_mode == -1) and (line.startswith('n')):
                img_id = (line.rstrip()).split('-')[0]
                if cur_mode==0 and img_id not in invalid:
                    f_invalid.write(cat+'/'+img_id+'.jpg\n')
                    invalid.append(img_id)
                elif cur_mode == 1 and img_id not in amb:
                    f_amb.write(cat+'/'+img_id+'.jpg\n')
                    amb.append(img_id)
                elif cur_mode == 2 and img_id not in cont:
                    f_cont.write(cat+'/'+img_id+'.jpg\n')
                    cont.append(img_id)
                elif cur_mode == 3 and img_id not in persp:
                    f_persp.write(cat+'/'+img_id+'.jpg\n')
                    persp.append(img_id)
        fr.close()
f_invalid.close()
f_amb.close()
f_cont.close()
f_persp.close()
print '/invalid.txt'

Generating invalid list of photos-- /invalid.txt


In [37]:
#generate image list
f = open(output_fold + '/image_list.txt','w')
for cat in cat_list:
    full_path = photo_path+'/'+cat
    for photo in os.listdir(full_path):
        rel_path = cat+'/'+photo
        f.write(rel_path+'\n')
f.close()

In [86]:
#load image list
with open(output_fold+'/image_list.txt','r') as my_file:
    img_list = [c.rstrip() for c in my_file.readlines()]

In [57]:
#save canny edge version of photos
canny_out = root_path+"canny/"

for path in img_list:
    full_path = photo_path + '/'+path
    img = color.rgb2gray(plt.imread(full_path.rstrip()));
    img = imresize(img,[256,256])
    canny = feature.canny(img,sigma=2)
    if not os.path.exists(canny_out+path.split('/')[0]):
        os.makedirs(canny_out+path.split('/')[0])
    imsave(canny_out+path.rstrip(),~canny)



In [59]:
#load invalid list 
with open(output_fold+'/invalid_photo/invalid.txt','r') as my_file:
    invalid = [c.rstrip() for c in my_file.readlines()]
with open(output_fold+'/invalid_photo/ambiguous.txt','r') as my_file:
    amb = [c.rstrip() for c in my_file.readlines()]
with open(output_fold+'/invalid_photo/context.txt','r') as my_file:
    cont = [c.rstrip() for c in my_file.readlines()]
with open(output_fold+'/invalid_photo/perspective.txt','r') as my_file:
    persp = [c.rstrip() for c in my_file.readlines()]

In [7]:
#create cat dict
index_of_cat = {}
for i,cat in enumerate(cat_list):
    index_of_cat[cat] = i

In [72]:
#check number of avaiable image in each category
cat_c = [0]*125
for path in img_list:
    cat = path.split('/')[0]
    if path not in invalid and path not in cont:
        cat_c[index_of_cat[cat]] =cat_c[index_of_cat[cat]]+1 
min(cat_c)

28

In [5]:
def checkFullSketches(path,root_folder):
    cat = path.split('/')[0]
    img_id = path.split('/')[1][:-4]
    sketch_list = os.listdir(root_path+'/sketches/'+cat)
    sketch_img_list = [skg for skg in sketch_list if skg.startswith(img_id+'-') and skg.endswith('-5.png')]
    if np.shape(sketch_img_list)[0] == 5:
        return True
    else:
        return False

In [9]:
#gen test set, ideally not in any of the invalid list
#damn, forgot to check 5 sketches
cat_c = [0]*125

f = open(output_fold+'/test_img_list.txt','w')
for path in np.random.permutation(img_list):
    cat = path.split('/')[0]
    if (checkFullSketches(path,root_path) or cat=='jack-o-lantern') and path not in invalid and path not in cont and cat_c[index_of_cat[cat]]<10:
        f.write(path+'\n')
        cat_c[index_of_cat[cat]] =cat_c[index_of_cat[cat]]+1 
f.close()

In [ ]:
path

In [93]:
with open(output_fold+'/test_img_list.txt','r') as my_file:
    test_img_list = [c.rstrip() for c in my_file.readlines()]

In [94]:
#gen train set, any leftover not in the test set

f = open(output_fold+'/train_img_list.txt','w')
cat_c = [0]*125
for path in img_list:
    if path not in test_img_list:
        cat = path.split('/')[0]
        f.write(path+'\n')
        cat_c[index_of_cat[cat]] =cat_c[index_of_cat[cat]]+1 
f.close()

In [95]:
with open(output_fold+'/train_img_list.txt','r') as my_file:
    train_img_list = [c.rstrip() for c in my_file.readlines()]